In [2]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 1.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=9478ec5c373dac38ae765655c751531fe4d3454ff9a87a9aa25f06a9d5ba0679
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=3f038de0f546dc084eec2c5de3be18afa503ce8bf1a50ec415170b5537d89492
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [4]:
fashion_mnist=keras.datasets.fashion_mnist
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [5]:
train_images=train_images/255.0
test_images=test_images/255.0
train_images[0].shape

(28, 28)

In [6]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [7]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [8]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 17s 10ms/step - loss: 0.5314 - accuracy: 0.8119 - val_loss: 0.4194 - val_accuracy: 0.8443
Epoch 2/3
1688/1688 [==============================] - 16s 9ms/step - loss: 0.3868 - accuracy: 0.8573 - val_loss: 0.3964 - val_accuracy: 0.8538
Epoch 3/3
1688/1688 [==============================] - 16s 9ms/step - loss: 0.3604 - accuracy: 0.8651 - val_loss: 0.4244 - val_accuracy: 0.8438


Epoch 1/3
1688/1688 [==============================] - 13s 8ms/step - loss: 0.4821 - accuracy: 0.8330 - val_loss: 0.3897 - val_accuracy: 0.8578
Epoch 2/3
1688/1688 [==============================] - 13s 8ms/step - loss: 0.3546 - accuracy: 0.8706 - val_loss: 0.3660 - val_accuracy: 0.8663
Epoch 3/3
1688/1688 [==============================] - 13s 8ms/step - loss: 0.3279 - accuracy: 0.8787 - val_loss: 0.4232 - val_accuracy: 0.8575


Epoch 1/3
1688/1688 [==============================] - 18s 11ms/step - loss: 0.3952 - accuracy: 0.8571 - val_loss: 0.2914 - val_accuracy: 0.8902
Epoch 2/3
1688/1688 [==============================] - 18s 11ms/step - loss: 0.2497 - accuracy: 0.9066 - val_loss: 0.2751 - val_accuracy: 0.9035
Epoch 3/3
1688/1688 [==============================] - 18s 10ms/step - loss: 0.1913 - accuracy: 0.9281 - val_loss: 0.2728 - val_accuracy: 0.9012


Epoch 1/3
1688/1688 [==============================] - 17s 10ms/step - loss: 0.4377 - accuracy: 0.8443 - val_loss: 0.3595 - val_accuracy: 0.8642
Epoch 2/3
1688/1688 [==============================] - 17s 10ms/step - loss: 0.3221 - accuracy: 0.8823 - val_loss: 0.3372 - val_accuracy: 0.8770
Epoch 3/3
1688/1688 [==============================] - 17s 10ms/step - loss: 0.2777 - accuracy: 0.8959 - val_loss: 0.3375 - val_accuracy: 0.8857


Epoch 1/3
1688/1688 [==============================] - 14s 8ms/step - loss: 0.5637 - accuracy: 0.8114 - val_loss: 0.4054 - val_accuracy: 0.8482
Epoch 2/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.4095 - accuracy: 0.8489 - val_loss: 0.4402 - val_accuracy: 0.8383
Epoch 3/3
1688/1688 [==============================] - 14s 8ms/step - loss: 0.3734 - accuracy: 0.8639 - val_loss: 0.3976 - val_accuracy: 0.8593


INFO:tensorflow:Oracle triggered exit


In [9]:
model=tuner_search.get_best_models(num_models=1)[0]

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 112)       2912      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        64576     
_________________________________________________________________
flatten (Flatten)            (None, 30976)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               3469424   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 3,538,042
Trainable params: 3,538,042
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 19s 11ms/step - loss: 0.1932 - accuracy: 0.9276 - val_loss: 0.2583 - val_accuracy: 0.9057
Epoch 5/10
1688/1688 [==============================] - 18s 11ms/step - loss: 0.1452 - accuracy: 0.9457 - val_loss: 0.2667 - val_accuracy: 0.9142
Epoch 6/10
1688/1688 [==============================] - 18s 11ms/step - loss: 0.1108 - accuracy: 0.9585 - val_loss: 0.2955 - val_accuracy: 0.9143
Epoch 7/10
1688/1688 [==============================] - 18s 11ms/step - loss: 0.0786 - accuracy: 0.9707 - val_loss: 0.3367 - val_accuracy: 0.9118
Epoch 8/10
1688/1688 [==============================] - 18s 11ms/step - loss: 0.0625 - accuracy: 0.9769 - val_loss: 0.4030 - val_accuracy: 0.9097
Epoch 9/10
1688/1688 [==============================] - 18s 11ms/step - loss: 0.0459 - accuracy: 0.9832 - val_loss: 0.4032 - val_accuracy: 0.9125
Epoch 10/10
1688/1688 [==============================] - 18s 11ms/step - loss: 0.0389 - accuracy: 0.9857 - val_loss: 0.5088 